In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp dialog_system

In [ ]:
# export
from let_me_answer_for_you.settings import *
from deeppavlov import train_model
import logging
from unittest.mock import patch
from collections import defaultdict

import pandas as pd 

logging.basicConfig(
    #filename='example.log',
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.ERROR,
    datefmt='%I:%M:%S'
)

logging.debug(" Debug Log Active")
logging.info("Hello! Welcome to our automated dialog system!")
logging.warning(' Warning Log Active')
#logging.error(' Error Log Active ')

# Dialog System Class
> Class that implements the `question_response`, `new_question_answer` and `new_context` methods

In [ ]:
#export
class DialogSystem:
    ''' The DialogSystem class implements the main methods
    defined in settings module. \n
    INPUT: \n
    - context_data_file: csv file of contexts (default: None)\n
    - faq_data_file: csv file of FAQs (default: None)\n
    - configs_faq: json config file (default: None)\n
    - download_models: Indicates if download configuration files (default: True)\n
    
    If the context or the faq files are not provided, a *data* directory with the missing files,
    will be created (in the same path where the module is running). \n
    When an instance is created, the 'run_shell_installs()' and 'load_and_prepare_data()' routines are called,
    also the *data* and *qa_models* attributes are created, they store the information of the dataframes
    and of the models, respectively.
    '''
    def __init__(
        self,
        context_data_file=None,
        faq_data_file=None,
        configs_faq=None,
        download_models=True
    ):
        ''' The init of the class  '''
        run_shell_installs()
        self.data = {'context': defaultdict(str), 'faq': defaultdict(str)}
        self.download = download_models
        load_and_prepare_data(
            context_data_file=context_data_file,
            faq_data_file=faq_data_file,
            configs_faq=configs_faq,
            data=self.data
        )
        self.qa_models = load_qa_models(
            config_tfidf=self.data['faq']['config'], download=self.download
        )

    def question_answer(self, question):
        ''' Gets answers to a question. \n
        INPUT: \n
        - *question* parameter \n
        The method creates the following attributes:\n
        - 'self.question' -> the question parameter \n
        - 'self.response' -> a list of possible responses \n
        - 'self.formatted_responses' -> string of the question-answer pair
        '''
        self.question = question
        self.responses, self.formatted_responses = get_responses(
            self.data['context']['df'],
            question,
            self.qa_models,
            nb_squad_results=1
        )

    def new_question_answer(self, question, answer):
        '''Adds a new question-answer pair.\n
        INPUT:\n
        - question\n
        - answer\n        
        
        The new question-answer pair is stored in the csv in the path *self.data['faq']['path']*
        '''
        _faq = self.data['faq']
        new_faq = pd.DataFrame({'Question': [question], 'Answer': [answer]})
        _faq['df'] = _faq['df'].append(new_faq)
        _faq['df'].to_csv(_faq['path'], index=False)
        self.qa_models['faq']['tfidf'] = train_model(
            _faq['config'], download=False
        )
        self.question, self.answer = question, answer
        logging.info('FAQ dataset and model updated..')

    def new_context(self, topic, context):
        ''' Adds a new context. \n
        INPUT:\n
        - topic (The title of the context)
        - context
        
        The new context is stored in the csv in the path *self.data['context']['path']*
        '''
        _ctx = self.data['context']
        new_context = pd.DataFrame({'topic': [topic], 'context': [context]})
        _ctx['df'] = _ctx['df'].append(new_context)
        _ctx['df'].to_csv(_ctx['path'], index=False)
        self.topic, self.context = topic, context
        logging.info('contexts dataset updated..')

In [ ]:
from nbdev.showdoc import *
show_doc(DialogSystem)
show_doc(DialogSystem.question_answer)
show_doc(DialogSystem.new_question_answer)
show_doc(DialogSystem.new_context)

<h2 id="DialogSystem" class="doc_header"><code>class</code> <code>DialogSystem</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DialogSystem</code>(**`context_data_file`**=*`None`*, **`faq_data_file`**=*`None`*, **`configs_faq`**=*`None`*, **`download_models`**=*`True`*)

The DialogSystem class implements the main methods
defined in settings module. 

INPUT: 

- context_data_file: csv file of contexts (default: None)

- faq_data_file: csv file of FAQs (default: None)

- configs_faq: json config file (default: None)

- download_models: Indicates if download configuration files (default: True)


If the context or the faq files are not provided, a *data* directory with the missing files,
will be created (in the same path where the module is running). 

When an instance is created, the 'run_shell_installs()' and 'load_and_prepare_data()' routines are called,
also the *data* and *qa_models* attributes are created, they store the information of the dataframes
and of the models, respectively.

<h4 id="DialogSystem.question_answer" class="doc_header"><code>DialogSystem.question_answer</code><a href="__main__.py#L38" class="source_link" style="float:right">[source]</a></h4>

> <code>DialogSystem.question_answer</code>(**`question`**)

Gets answers to a question. 

INPUT: 

- *question* parameter 

The method creates the following attributes:

- 'self.question' -> the question parameter 

- 'self.response' -> a list of possible responses 

- 'self.formatted_responses' -> string of the question-answer pair

<h4 id="DialogSystem.new_question_answer" class="doc_header"><code>DialogSystem.new_question_answer</code><a href="__main__.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>DialogSystem.new_question_answer</code>(**`question`**, **`answer`**)

Adds a new question-answer pair.

INPUT:

- question

- answer


The new question-answer pair is stored in the csv in the path *self.data['faq']['path']*

<h4 id="DialogSystem.new_context" class="doc_header"><code>DialogSystem.new_context</code><a href="__main__.py#L73" class="source_link" style="float:right">[source]</a></h4>

> <code>DialogSystem.new_context</code>(**`topic`**, **`context`**)

Adds a new context. 

INPUT:

- topic (The title of the context)
- context

The new context is stored in the csv in the path *self.data['context']['path']*

## Test Examples 

In [ ]:
ds = DialogSystem(
    faq_data_file='./faq_example.csv',
    context_data_file='./context_example.csv',
    download_models=False
)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
ds.question_answer(question = 'What is Intekglobal?')

assert ds.question == 'What is Intekglobal?'
assert ds.responses == ['This is the Intekglobal Dialog System']
assert ds.formatted_responses == 'What is Intekglobal?:\n\n1: This is the Intekglobal Dialog System\n'

In [ ]:
ds.new_question_answer(question='What day is today?', answer='Today is the day!')
ds.question_answer(question='What day is today?')

assert ds.formatted_responses == 'What day is today?:\n\n1: Today is the day!\n'

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
context = '''Space Exploration Technologies Corp., trading as SpaceX,
is an American aerospace manufacturer and space transportation services company headquartered
in Hawthorne, California. It was founded in 2002 by Elon Musk with the goal of reducing 
space transportation costs to enable the colonization of Mars. SpaceX has developed several launch 
vehicles,the Starlink satellite constellation, and the Dragon spacecraft.
'''
ds.new_context(topic='SpaceX', context=context)
ds.question_answer(question='What are SpaceX initials stand for?')

assert ds.responses == [
    'Space Exploration Technologies Corp.',
    'Space Exploration Technologies Corp.', 'Today is the day!'
]

In [ ]:
#hide
from shutil import rmtree
from os import remove
remove('./faq_example.csv')
remove('./context_example.csv')
rmtree('./data', ignore_errors=True)